In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import percentileofscore
import sys
from tqdm import tqdm_notebook
sys.path.append('../')

%matplotlib inline

In [23]:
from xgboost.sklearn import XGBClassifier

In [24]:
xgb = XGBClassifier()

In [42]:
xgb.fit(df[oof_cols], df['TARGET'], eval_set=[(df[oof_cols], df['TARGET']), (df[oof_cols], df['TARGET'])], eval_metric='auc', verbose=10)

[0]	validation_0-auc:0.791452	validation_1-auc:0.791452
[10]	validation_0-auc:0.800144	validation_1-auc:0.800144
[20]	validation_0-auc:0.801067	validation_1-auc:0.801067
[30]	validation_0-auc:0.801346	validation_1-auc:0.801346
[40]	validation_0-auc:0.801501	validation_1-auc:0.801501
[50]	validation_0-auc:0.801743	validation_1-auc:0.801743
[60]	validation_0-auc:0.802075	validation_1-auc:0.802075
[70]	validation_0-auc:0.802363	validation_1-auc:0.802363
[80]	validation_0-auc:0.802493	validation_1-auc:0.802493
[90]	validation_0-auc:0.802663	validation_1-auc:0.802663
[99]	validation_0-auc:0.802767	validation_1-auc:0.802767


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [40]:
xgb.feature_importances_

array([0.10240964, 0.1370482 , 0.15813252, 0.21084337, 0.0813253 ,
       0.09789157, 0.2123494 ], dtype=float32)

In [43]:
xgb.evals_result_

{'validation_0': {'auc': [0.791452,
   0.795324,
   0.797328,
   0.797478,
   0.797989,
   0.799331,
   0.799511,
   0.799781,
   0.799894,
   0.800059,
   0.800144,
   0.800185,
   0.800246,
   0.800757,
   0.800768,
   0.800775,
   0.800805,
   0.800995,
   0.801031,
   0.801044,
   0.801067,
   0.801069,
   0.801095,
   0.801135,
   0.801134,
   0.801153,
   0.801168,
   0.801315,
   0.801315,
   0.801319,
   0.801346,
   0.801356,
   0.801359,
   0.801379,
   0.801384,
   0.801404,
   0.801415,
   0.801462,
   0.801472,
   0.801498,
   0.801501,
   0.801514,
   0.801528,
   0.801551,
   0.801564,
   0.801608,
   0.801618,
   0.801654,
   0.801676,
   0.801712,
   0.801743,
   0.801796,
   0.801814,
   0.801841,
   0.801867,
   0.801898,
   0.801937,
   0.801968,
   0.802003,
   0.802039,
   0.802075,
   0.8021,
   0.802107,
   0.802133,
   0.802163,
   0.802214,
   0.802249,
   0.802269,
   0.802277,
   0.80236,
   0.802363,
   0.802363,
   0.802426,
   0.802427,
   0.802444,
   0.

In [38]:
xgb.evals_result()

{'validation_0': {'auc': [0.791452,
   0.795324,
   0.797328,
   0.797478,
   0.797989,
   0.799331,
   0.799511,
   0.799781,
   0.799894,
   0.800059,
   0.800144,
   0.800185,
   0.800246,
   0.800757,
   0.800768,
   0.800775,
   0.800805,
   0.800995,
   0.801031,
   0.801044,
   0.801067,
   0.801069,
   0.801095,
   0.801135,
   0.801134,
   0.801153,
   0.801168,
   0.801315,
   0.801315,
   0.801319,
   0.801346,
   0.801356,
   0.801359,
   0.801379,
   0.801384,
   0.801404,
   0.801415,
   0.801462,
   0.801472,
   0.801498,
   0.801501,
   0.801514,
   0.801528,
   0.801551,
   0.801564,
   0.801608,
   0.801618,
   0.801654,
   0.801676,
   0.801712,
   0.801743,
   0.801796,
   0.801814,
   0.801841,
   0.801867,
   0.801898,
   0.801937,
   0.801968,
   0.802003,
   0.802039,
   0.802075,
   0.8021,
   0.802107,
   0.802133,
   0.802163,
   0.802214,
   0.802249,
   0.802269,
   0.802277,
   0.80236,
   0.802363,
   0.802363,
   0.802426,
   0.802427,
   0.802444,
   0.

In [2]:
pred_train = pd.read_csv('../workdir/ScaleWeight_2_Seed_1024/preds.csv')

In [3]:
baseModelNames = [
    'LB0.802',
    'allParamsTuned',
    'withAggFeatsIn',
    'PrunedFeatureModel',
    '0824_new_agg_Final',
    'WithPosScaleWeight_eq_5',
    '10CV',
]

In [4]:
def get_data(modelNames):
    oof = {}
    preds = {}
    for name in tqdm_notebook(modelNames):
        pathName = '../workdir/' + name
        with open(pathName + '/kfold_lgb.pkl', 'rb') as f:
            lgbClf = pickle.load(f)
        oof_arr = lgbClf.oof_preds_
        pred_arr = pd.read_csv(pathName + '/preds.csv', usecols=['TARGET']).values.ravel()

        oof[name + '_oof'] = oof_arr
        preds[name + '_preds'] = pred_arr

    return oof, preds

oof, preds = get_data(baseModelNames)
df = pd.DataFrame(oof)
predDf = pd.DataFrame(preds)

A Jupyter Widget

In [5]:
df = df.join(pd.read_csv('../input/train_df.csv', usecols=['SK_ID_CURR', 'TARGET']))
oof_cols = [col for col in df.columns if '_oof' in col]
# df[oof_cols] = df[oof_cols].apply(lambda x: x.rank() / len(x))
df['std'] = df[oof_cols].std(axis=1)

In [6]:
predDf = predDf.join(pd.read_csv('../input/sample_submission.csv', usecols=['SK_ID_CURR']))
pred_cols = [col for col in predDf.columns if '_preds' in col]
predDf['std'] = predDf[pred_cols].std(axis=1)

<font size=5> OOF AUC

In [7]:
for col in oof_cols:
    auc = roc_auc_score(df['TARGET'], df[col])
    print('%-30s    %.6f' % (col, auc))

LB0.802_oof                       0.798360
allParamsTuned_oof                0.797814
withAggFeatsIn_oof                0.798296
PrunedFeatureModel_oof            0.796308
0824_new_agg_Final_oof            0.798611
WithPosScaleWeight_eq_5_oof       0.797380
10CV_oof                          0.799363


<font size=5> Correlation

In [22]:
def avg_score(weights, df, cols):
    ret = 0
    for i, col in enumerate(cols):
        ret += weights[i] * df[col]
    return ret
avg_score_ = avg_score([0.2, 0, 0.1, 0.2, 0, 0, 0.5], df, oof_cols)
roc_auc_score(df['TARGET'], avg_score_)

0.8002229989334002

In [10]:
avg_score_ = avg_score([0.2, 0.2, 0.2, 0.2, 0.2, 0, 0.2], predDf, pred_cols)

NameError: name 'avg_score' is not defined

In [ ]:
predDf['TARGET'] = avg_score_

In [ ]:
predDf

In [ ]:
predDf[['SK_ID_CURR', 'TARGET']].to_csv('weighted_avg.csv', index=False)

In [8]:
df[oof_cols].describe()

,LB0.802_oof,allParamsTuned_oof,withAggFeatsIn_oof,PrunedFeatureModel_oof,0824_new_agg_Final_oof,WithPosScaleWeight_eq_5_oof,10CV_oof
count,307507.000000,307507.000000,307507.000000,307507.000000,307507.000000,307507.000000,307507.000000
mean,0.078900,0.079093,0.078945,0.078977,0.078994,0.223849,0.078928
std,0.095012,0.094832,0.095678,0.096303,0.095510,0.182805,0.095482
min,0.001466,0.001575,0.001279,0.001190,0.001302,0.004251,0.001443
25%,0.021588,0.021691,0.021235,0.020750,0.021344,0.083227,0.021448
50%,0.043870,0.044081,0.043577,0.042969,0.043726,0.163316,0.043679
75%,0.095396,0.095970,0.095571,0.095740,0.095522,0.316467,0.095516
max,0.890341,0.899455,0.879603,0.878445,0.883344,0.955054,0.900269


In [9]:
df[oof_cols].corr()

,LB0.802_oof,allParamsTuned_oof,withAggFeatsIn_oof,PrunedFeatureModel_oof,0824_new_agg_Final_oof,WithPosScaleWeight_eq_5_oof,10CV_oof
LB0.802_oof,1.000000,0.976576,0.984745,0.967766,0.987605,0.939105,0.986135
allParamsTuned_oof,0.976576,1.000000,0.977530,0.961129,0.979595,0.927106,0.978974
withAggFeatsIn_oof,0.984745,0.977530,1.000000,0.971140,0.988000,0.931213,0.977416
PrunedFeatureModel_oof,0.967766,0.961129,0.971140,1.000000,0.970998,0.920206,0.961152
0824_new_agg_Final_oof,0.987605,0.979595,0.988000,0.970998,1.000000,0.933923,0.980044
WithPosScaleWeight_eq_5_oof,0.939105,0.927106,0.931213,0.920206,0.933923,1.000000,0.932281
10CV_oof,0.986135,0.978974,0.977416,0.961152,0.980044,0.932281,1.000000


<font size=5> start Stacking

In [ ]:
def train(X, y, model, cv=5):
    from copy import deepcopy
    models = [deepcopy(model) for i in range(cv)]
    split = StratifiedKFold(n_splits=cv, shuffle=True)
    oof_preds = np.zeros((len(X),))
    for i, (train_idx, dev_idx) in enumerate(split.split(X, y)):
        X_train, X_dev = X.iloc[train_idx], X.iloc[dev_idx]
        y_train, y_dev = y.iloc[train_idx], y.iloc[dev_idx]
        
        model = models[i]
        print('start train fold %s' % str(i + 1))
        model.fit(X_train, y_train,
#                   eval_set=[(X_train, y_train), (X_dev, y_dev)],
#                   eval_names=['train', 'valid'],
#                   eval_metric='auc',
#                   verbose=20,
#                   early_stopping_rounds=50,
                 )
        y_hat = model.predict_proba(X_dev)[:, 1]
        oof_preds[dev_idx] = y_hat
        print('=' * 60)
    print("FULL AUC: ", roc_auc_score(y, oof_preds))
    return models

# lgbClf = LGBMClassifier(n_estimators=250, learning_rate=0.01, reg_alpha=1)
models = train(df[oof_cols] , df['TARGET'], LogisticRegression())

In [ ]:
for model in models:
    print(model.coef_)

In [ ]:
def pred(X, models):
    y_hat = np.zeros((len(X), ))
    n = len(models)
    for model in models:
        y_hat += model.predict_proba(X)[:, 1]
    y_hat /= n
    return y_hat

stackingPreds = pred(predDf[pred_cols], models)